In [146]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_All_b.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

,document,link,label,doc_len
0,대선에 출마하는 국회의원은 12월까지 자리에서 물러나야 한다,https://factcheck.snu.ac.kr/v2/facts/3191,0,7
1,노령연금을 더 빨리 또는 더 늦게 받을 수 있다?,https://factcheck.snu.ac.kr/v2/facts/964,1,9
2,"""문 정부 탈원전 정책이 산불 원인이다""",https://factcheck.snu.ac.kr/v2/facts/1499,0,6
3,"해병대, 육·해·공군 보다 구타·가혹행위 심하다?",https://factcheck.snu.ac.kr/v2/facts/414,1,5
4,"""언론이 정신질환에 대한 사회적 혐오를 부추겨왔다""",https://factcheck.snu.ac.kr/v2/facts/1655,1,7
...,...,...,...,...
1436,"""공정거래위원회 전속고발권 없애면 검찰권한 확대된다.""",https://factcheck.snu.ac.kr/v2/facts/2697,0,5
1437,"""박근혜 정부 때 카풀법안 허용""?",https://factcheck.snu.ac.kr/v2/facts/1245,0,5
1438,“대외경제정책연구원의 유럽 사무소 설치 예산을 전액 삭감했다”,https://factcheck.snu.ac.kr/v2/facts/662,0,7
1439,"""연동형 비례대표제는 위헌이다""",https://factcheck.snu.ac.kr/v2/facts/1909,0,4


In [147]:
# train_df['label'] = train_df['label'].map({'전혀 사실 아님':0, '대체로 사실 아님':0, '사실':1, '대체로 사실':1})
# train_df.dropna(axis=0, inplace=True)
# train_df.reset_index(drop=True, inplace=True)
# train_df['label'].astype(int)
# train_df

**데이터 불균형 -> Under Sampling**

In [148]:
# train_df_true = train_df[train_df['label'] == 1]
# train_df_true.reset_index(drop=True, inplace=True)
# train_df_fake = train_df[train_df['label'] == 0]
# train_df_fake.reset_index(drop=True, inplace=True)

In [149]:
# train_df_true.shape

In [150]:
# train_df_fake_sample = train_df_fake.loc[0:255]
# train_df_fake_sample

In [151]:
# train_df_balance = pd.concat([train_df_true, train_df_fake_sample])
# train_df_balance = train_df_balance.sample(frac=1).reset_index(drop=True)
# train_df_balance

In [152]:
# train_df = train_df_balance

In [153]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import boto3
import sys
from gensim.models import FastText
import os, csv, math, codecs
from tqdm import tqdm
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


MAX_NB_WORDS = 100000

# =========================================================================
# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words

# =========================================================================

embeddings_index = {}
f = codecs.open('/home/ubuntu/FastText/wiki.ko.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

# ==================================================================================
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

my_Tokenizer = MyTokenizer(Okt())

raw_docs_train = train_df['document'].tolist()
# raw_docs_test = test_df['document'].tolist()
# print(raw_docs_test)
num_classes = len(label_names)

processed_docs_train = []
# processed_docs_test = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

# for doc in tqdm(raw_docs_test):
#     tokens = my_Tokenizer(doc)
#     processed_docs_test.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
# tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)
tokenizer.fit_on_texts(processed_docs_train)
# tokenizer.fit_on_texts(processed_docs_train+proccessed_docs_test)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
# word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size : ", len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
# word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

y_train = train_df['label'].values
# ====================================================================================
# training params
batch_size = 256
num_epochs = 40

# model parameters
num_filters = 64
embed_dim = 300
weight_decay = 1e-4

# embedding matrix

words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))

# print(word_index)
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
        # cc.ko.300.vec에서 찾지 못한 단어들의 리스트.
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))



879131it [02:09, 6766.10it/s]
  6%|▌         | 84/1441 [00:00<00:01, 838.87it/s]

found 879130 word vectors


100%|██████████| 1441/1441 [00:03<00:00, 369.57it/s]


dictionary size :  3974
number of null word embeddings: 349
sample words not found:  ['25' '150만원' '50%' '8000' '공시가격' '2년' '2~3년' '200만' '펼쳐졌습니다' '2014년']


In [154]:
# ========================================================================================
from keras.layers import BatchNormalization
import tensorflow as tf

model = tf.keras.Sequential()

model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))

model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))
model.summary()
# ================================================================================
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

es_callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(word_seq_train, y_train, batch_size=32,
          epochs=num_epochs, 
                    validation_split=0.2, 
                    callbacks=[es_callback], 
                    shuffle=False)

# predictions = model.predict_classes(word_seq_test)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 11, 300)           1192500   
_________________________________________________________________
dropout_20 (Dropout)         (None, 11, 300)           0         
_________________________________________________________________
dense_30 (Dense)             (None, 11, 32)            9632      
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 11, 128)           49664     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 64)                41216     
_________________________________________________________________
dense_31 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)              

In [155]:
model.save('SNU_LSTM_Model_All_balance.h5')

In [34]:
balanced_model = load_model('SNU_LSTM_Model_경제_balance.h5')
balanced_score = balanced_model.evaluate(word_seq_train, y_train, batch_size=1)
model = load_model('SNU_LSTM_Model_경제.h5')
scores = model.evaluate(word_seq_train, y_train, batch_size=1)
all_model = load_model('SNU_LSTM_Model_All.h5')
all_scores = all_model.evaluate(word_seq_train, y_train, batch_size=1)

197/197 [==============================] - 7s 6ms/step - loss: 0.9111 - accuracy: 0.4975


In [44]:
print("경제 balanced data LSTM %s: %.2f%%" %(balanced_model.metrics_names[1], scores[1]*100))
print("경제 data LSTM %s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
print("전체 data LSTM %s: %.2f%%" %(all_model.metrics_names[1], all_scores[1]*100))

경제 balanced data LSTM accuracy: 50.76%
경제 data LSTM accuracy: 50.76%
전체 data LSTM accuracy: 49.75%
